In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from vastai.api import VastClient
from io import StringIO
import time
from IPython.display import clear_output

In [4]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


In [5]:
def monitor_file(log_file, popen=None, n_lines=5, update_freq_s=1):
    while True:
        try:
            clear_output(wait=True)
            print(remote['tail'](log_file,'-n%i'%n_lines))
            if popen is not None:
                ret = popen.proc.poll()
                if ret is not None:
                    print("Process has exited with code:",popen.proc.poll())
                    break
            time.sleep(update_freq_s)
        except KeyboardInterrupt:
            break
        except NameError:
            break

In [31]:
instance = vast.get_running_instances()[0]
remote = instance.pb_remote
instance

405814: running   running              $0.0181/hr ssh4.vast.ai:15814  177.5↑ 245.7↓  flops:2.46144T
	1X 2.0GB GTX 1050, 32.0/32X 31.6GB Xeon® Silver 4109T 
	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter

## Check mongo status

In [32]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n5'))

2019-11-05T06:45:35.297+0000 I NETWORK  [initandlisten] waiting for connections on port 27017
2019-11-05T06:50:35.297+0000 I STORAGE  [thread1] createCollection: config.system.sessions with generated UUID: 2952b267-b50f-49e1-80a3-08c25e6f91de
2019-11-05T06:50:35.317+0000 I INDEX    [thread1] build index on: config.system.sessions properties: { v: 2, key: { lastUse: 1 }, name: "lsidTTLIndex", ns: "config.system.sessions", expireAfterSeconds: 1800 }
2019-11-05T06:50:35.317+0000 I INDEX    [thread1] 	 building index using bulk method; build may temporarily use up to 500 megabytes of RAM
2019-11-05T06:50:35.320+0000 I INDEX    [thread1] build index done.  scanned 0 total records. 0 secs



In [33]:
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 15814 -i /home/sam/.ssh/vastai


## Install model and download data

In [14]:
install_script_target='/root/install_script.sh'
install_script = StringIO("""#!/bin/sh
INSTALL_LOG=/root/install.log
echo avian_vocalizations INSTALL BEGIN `date` >> $INSTALL_LOG 2>&1
cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git  >> $INSTALL_LOG 2>&1
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pip install pipenv  >> $INSTALL_LOG 2>&1
pipenv lock -r > requirements.txt
pip install -r requirements.txt >> $INSTALL_LOG 2>&1
pip install -e . >> $INSTALL_LOG 2>&1
download_data >> $INSTALL_LOG 2>&1
echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
""")
# ("""pipenv install >> $INSTALL_LOG 2>&1
# pipenv run python setup.py develop >> $INSTALL_LOG 2>&1
# pipenv run download_data >> $INSTALL_LOG 2>&1
# echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
# """)
# monitor_output(mongo_instance, install_script)
remote.sftp.putfo(install_script,install_script_target)
remote['chmod']('+x',install_script_target)
p=remote.session().popen(install_script_target)
p

In [ ]:
monitor_file('/root/install.log', p)

## Start hyperopt worker

In [88]:
hyperopt_worker_log="/root/hyperopt_worker.log"
start_hyperopt_worker_target='/root/avian_vocalizations/start_hyperopt_worker.sh'
start_hyperopt_worker="#!/bin/sh\ncd /root/avian_vocalizations\n"\
    "export LC_ALL=C.UTF-8\n"\
    "export LANG=C.UTF-8\n"\
    "hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations "\
    " --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=."\
    " >> %s 2>&1\n"%hyperopt_worker_log

# print(start_hyperopt_worker)
remote.sftp.putfo(StringIO(start_hyperopt_worker),start_hyperopt_worker_target)
remote['chmod']('+x',start_hyperopt_worker_target)
pworker=remote.session().popen(start_hyperopt_worker_target)

In [89]:
monitor_file(hyperopt_worker_log, pworker)

  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node mfcc_conv2d_1_1/convolution}}]]
0 successful operations.
0 derived errors ignored.
INFO:hyperopt.mongoexp:exiting with N=9223372036854775806 after 1 consecutive exceptions

Process has exited with code: 0


In [ ]:
# vast.stop_all_instances()

In [62]:
list(remote.pgrep('hyperopt'))

[]